In [22]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=1000
println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    v+=star_algebra.(v)
    v=v./2
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=20
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p]
f=sum(vecf)

g=[1.0-sum(x[I[j]].^2) for j in 1:p]
J=[j:j for j in 1:p]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{false,Float64}}(undef,l)

randx=2*rand(Float64,n).-1
for j in 1:p
    randx[I[j]]=randx[I[j]]./norm(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=1000
Number of inequality constraints: m=51
Number of equality constraints: l=143


In [23]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [27]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=1

println("Term sparse order: t=",t)

Relaxed order: k=2
Term sparse order: t=1


In [28]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=5e-3,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[21, 20]
[49, 1]
------------------------------------------------------
  Number of cliques: p=50
  Largest clique size: u=21


  Largest constant trace: a=5.0
  Number of blocks: omega=202
  Number of equality consraints: zeta=67102
  Size of the largest block: s^max=442
Modeling time:
412.428809 seconds (2.11 G allocations: 283.123 GiB, 50.11% gc time)
iter=1   val=-508.76249317517966   gap=1.1150961402975128e-16   feas=30.226251473251985
iter=2   val=-454.95614732400645   gap=0.1429032460262949   feas=27.19436165017676
iter=4   val=-422.6777484486767   gap=0.2588745388547638   feas=17.99846574370734
iter=8   val=-262.3684127014755   gap=0.11664878878185153   feas=8.531178684906346
iter=16   val=-220.76788251387777   gap=0.017149351584980682   feas=4.917144736350016
iter=32   val=-195.48940227079936   gap=0.04138210877582374   feas=2.482422714395859
iter=64   val=-191.62012442367305   gap=0.005329213475754247   feas=1.7540160161978489
iter=128   val=-180.9287429587122   gap=0.009071587062728474   feas=1.1683458543012546
iter=256   val=-169.23251246599614   gap=0.0009620123085111211   feas=0.5857691747291222
i

-159.73962175602716

In [26]:
using NCTSSOS

@time begin

j=1
opt,data=cs_nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
        n,k,[dg;dh],numeq=l,minimize=true,obj="eigen",TS="block",QUIET=false);
while j <= t-1
    opt,data=cs_nctssos_higher!(data,TS="MD")
    j+=1
end
    
end

------------------------------------------------------
The clique sizes of varibles:
[21, 20]
[49, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 11481           
  Cones                  : 0               
  Scalar variables       : 245             
  Matrix variables       : 50              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Tim